# Graph Embedding for Amazon Recommendation System

In [71]:
import pandas as pd
import numpy as np
import re

from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds, norm
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

import operator
from collections import defaultdict

import networkx as nx
import node2vec 
from gensim.models import Word2Vec

In [14]:
data_path = "./ratings_books.csv"

# Data Loading

In [65]:
%%time
# open file to read amazon product metadata 
fhr = open('./amazon-meta.txt', 'r', encoding='utf-8', errors='ignore')

# initialize a nested product dictionary that will hold cleaned up amazon product data
# key = ASIN; value = MetaData associated with ASIN
amazonProducts = {}

# read the data from the amazon-meta file;
# populate amazonProducts nested dicitonary;
ct=0

(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", list(), "", list(), 0, 0, 0.0)
(time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull) = (list(),list(),list(),list(),list())
for line in fhr:
    if ct % 100000 == 0:
        print("{} Done!".format(ct))
    ct+=1
    
    line = line.strip()
    # a product block started
    if(line.startswith("Id")):

        line = line.replace(" ","")
        Id = line.split(":")[1]
    
    elif(line.startswith("ASIN")):

        line = line.replace(" ","")
        ASIN = line.split(":")[1]
     
    elif(line.startswith("title")):
 
        Title = line.split(":")[1]
  
        Title = ' '.join(Title.split())
    elif(line.startswith("group")):
        Group = line.split(":")[1]
 
    elif(line.startswith("salesrank")):
        line = line.replace(" ","")
        SalesRank = line.split(":")[1]

    elif(line.startswith("similar")):
        ls = line.split()
        Copurchased = ([c for c in ls[2:]])
        
    elif(line.startswith("categories")):
        ls = line.split()
        cat_list = []
        for i in range(int(ls[1].strip())):
            temp = fhr.readline()
            temp = temp.lower()
            temp = temp.replace("|","-")
            temp = temp[4:]
            temp = temp.strip()
            cat_list.append(temp)
        
        Categories = cat_list

    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
        l1,l2,l3,l4,l5=[],[],[],[],[]
        for i in range(int(ls[4].strip())):
            temp = fhr.readline()
            temp = temp.split()
            l1.append(temp[0])
            l2.append(temp[2])
            l3.append(temp[4])
            l4.append(temp[6])
            l5.append(temp[8])
        (time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull)=(l1,l2,l3,l4,l5)
        
            
    # a product block ended
    # write out fields to amazonProducts Dictionary
    elif (line==""):
        try:
            MetaData = {}
            if (ASIN != ""):
                amazonProducts[ASIN]=MetaData
            MetaData['Id'] = Id            
            MetaData['title'] = Title
            MetaData['categories'] = Categories
            MetaData['group'] = Group
            MetaData['co-purchased'] = Copurchased
            MetaData['sales_rank'] = int(SalesRank)
            MetaData['total_reviews'] = int(TotalReviews)
            MetaData['avg_rating'] = float(AvgRating)
            MetaData['review_time_stamps_list']    = time_stamp
            MetaData['review_ASINs_list']          = review_ASINs
            MetaData['review_ratings_list']        = review_ratings
            MetaData['review_votes_list']          = review_votes
            MetaData['review_found_helpfull_list'] = review_helpfull
            
            
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", list(), "", list(), 0, 0,0.0)
        (time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull) = (list(),list(),list(),list(),list())
fhr.close()

0 Done!
100000 Done!
200000 Done!
300000 Done!
400000 Done!
500000 Done!
600000 Done!
700000 Done!
800000 Done!
900000 Done!
1000000 Done!
1100000 Done!
1200000 Done!
1300000 Done!
1400000 Done!
1500000 Done!
1600000 Done!
1700000 Done!
1800000 Done!
1900000 Done!
2000000 Done!
2100000 Done!
2200000 Done!
2300000 Done!
2400000 Done!
2500000 Done!
2600000 Done!
2700000 Done!
2800000 Done!
2900000 Done!
3000000 Done!
3100000 Done!
3200000 Done!
3300000 Done!
3400000 Done!
3500000 Done!
3600000 Done!
3700000 Done!
3800000 Done!
3900000 Done!
4000000 Done!
4100000 Done!
4200000 Done!
4300000 Done!
4400000 Done!
4500000 Done!
4600000 Done!
4700000 Done!
4800000 Done!
4900000 Done!
CPU times: user 17.1 s, sys: 1.47 s, total: 18.6 s
Wall time: 19.6 s


In [66]:
%%time
df = pd.DataFrame(amazonProducts)
df = df.transpose()
df.shape

CPU times: user 8.07 s, sys: 558 ms, total: 8.62 s
Wall time: 8.66 s


(548552, 13)

In [68]:
temp = list(df['Id'].values)
df['Id'] = df.index
df.rename(columns={"Id":"ASIN"},inplace=True)
df.index = temp
data = df.copy()

def parse_date(date_list):
    new_list=[]
    for date in date_list:
        x = date.split("-")
        year,month,day = int(x[0]),int(x[1]),int(x[2])
        new_list.append((year,month,day))
    return new_list

def clean_category(cat_id_list):
    new_list=[]
    for row  in cat_id_list:
        temp=[]
        x = row.split("-")
        for cid in x:
            if "[" in cid:
                y = cid.split("[")
                cat  = y[0]
                ID  = (y[1][:-1])
            else:
                cat = cid
                ID = -1
            temp.append((cat,ID))
        new_list.append(temp)
    return new_list

In [72]:
%%time
df['ASIN']  = df['ASIN'].astype('str')
df['title'] = df['title'].apply(lambda x:re.sub(r'[^a-zA-Z0-9 ]','',x))
df['title'] = df['title'].str.lower()
df['categories'] = df['categories'].apply(clean_category)
df['group'] = df['group'].str.lower()
df['co-purchased']= df['co-purchased'].apply(lambda x: [str(i) for i in x])
df['sales_rank'] = df['sales_rank'].astype('int')
df['total_reviews'] = df['total_reviews'].astype('int')
df['avg_rating'] = df['avg_rating'].astype('float')
df['review_time_stamps_list'] = df['review_time_stamps_list'].apply(parse_date)
df['review_ASINs_list']= df['review_ASINs_list'].apply(lambda x: [str(i) for i in x])
df['review_ratings_list']= df['review_ratings_list'].apply(lambda x: [int(i) for i in x])
df['review_votes_list']= df['review_votes_list'].apply(lambda x: [int(i) for i in x])
df['review_found_helpfull_list']= df['review_found_helpfull_list'].apply(lambda x: [int(i) for i in x])

CPU times: user 20.2 s, sys: 29.1 s, total: 49.4 s
Wall time: 1min 5s


In [92]:
df_book = df.loc[df['group']==' book']

node_list = set(df_book['ASIN'])
amazonBooks = {}
for row in df_book[['ASIN','co-purchased']].iterrows():
        source_node= row[1][0]
        amazonBooks[source_node] = {}
        co_prod = row[1][1]
        amazonBooks[source_node] = ([cp for cp in co_prod if cp in node_list])

In [93]:
amazonBooks

{'0827229534': ['0804215715',
  '156101074X',
  '0687023955',
  '0687074231',
  '082721619X'],
 '0738700797': ['0738700827',
  '1567184960',
  '1567182836',
  '0738700525',
  '0738700940'],
 '0486287785': [],
 '0842328327': ['0842328130', '0842330313', '0842328610', '0842328572'],
 '1577943082': ['157794349X', '0892749504', '1577941829', '0892749563'],
 '0486220125': ['0486401960', '0452283612', '0486229076', '0714840343'],
 '0231118597': ['0375727191', '080148605X', '1560232579', '0300089023'],
 '1859677800': [],
 '0375709363': ['039474067X',
  '0679730672',
  '0679750541',
  '1400030668',
  '0896086704'],
 '0871318237': ['0060984341', '0553577514', '1571742972', '0962741817'],
 '1590770218': ['0871319640'],
 '0313230269': [],
 '1559362022': ['1559360968',
  '1559361247',
  '1559360828',
  '1559361018',
  '0743214552'],
 '0195110382': ['1585741485', '0140246967', '1557504288'],
 '0849311012': ['1580531784', '1578200326'],
 '078510870X': ['078511078X'],
 '3895780812': ['0865778973',
  

In [94]:
copurchase = []
for key in amazonBooks.keys():
    copurchase.append([key,amazonBooks[key]])

copurchase_data = pd.DataFrame(copurchase)
copurchase_data.rename(columns={0: "ASIN", 1: "Copurchase"})

,ASIN,Copurchase
0,0827229534,"[0804215715, 156101074X, 0687023955, 068707423..."
1,0738700797,"[0738700827, 1567184960, 1567182836, 073870052..."
2,0486287785,[]
3,0842328327,"[0842328130, 0842330313, 0842328610, 0842328572]"
4,1577943082,"[157794349X, 0892749504, 1577941829, 0892749563]"
...,...,...
393556,9700507734,[]
393557,9627762644,[]
393558,0970020503,[]
393559,1930519206,[]


## Create Co-Purchasing Graph

In [95]:
copurchaseGraph = nx.DiGraph(amazonBooks)
userid = set()
total_userid = df['review_ASINs_list'].values

In [96]:
copurchaseGraph = nx.Graph()
for index, row in df.iterrows():
  asin=row['ASIN']
  #print(asin)
  if asin not in copurchaseGraph.nodes():
    copurchaseGraph.add_node(asin)
  for a in row['co-purchased']:
    if a not in copurchaseGraph.nodes():
      copurchaseGraph.add_node(a)
    if not copurchaseGraph.has_edge(asin, a):
      copurchaseGraph.add_edge(asin, a)

## Graph Embedding Using Node2Vec

### Create a user-item graph with edge weights as the ratings.

In [3]:
data_user_item = pd.read_csv(data_path)

In [4]:
data_user_item.head()

,userID,productID,Rating
0,A2G90J4PELWGMN,0767905288,3
1,A93ZSJWZ4375R,0767905288,5
2,A3VZH0PWLQ9BB1,0767905288,4
3,AGLHUZX7082J9,0767905288,5
4,A2H27QB13MDOLB,0767905288,5


In [6]:
user2dict = dict()
product2dict = dict()
count = 0
for x in data_user_item.values:
    user = (x[0], 'userID')
    product = (x[1], 'productID')
    if user in user2dict:
        pass
    else:
        user2dict[user] = count
        count += 1
    if product in product2dict:
        pass
    else:
        product2dict[product] = count
        count += 1

In [7]:
len(user2dict), len(product2dict)

(34733, 7112)

In [8]:
user_product_graph = nx.Graph()

In [9]:
for x in data_user_item.values:
    user = (x[0], 'userID')
    product = (x[1], 'productID')
    user_product_graph.add_node(user2dict[user])
    user_product_graph.add_node(product2dict[product])
    user_product_graph.add_edge(user2dict[user], product2dict[product], weight=float(x[2]))

In [10]:
user_product_graph.number_of_edges()

763975

In [11]:
user_product_graph.number_of_nodes()

41845

We will use the implementation of DeepWalk provided in node2vec which is a bit different from original DeepWalk e.g. it uses negative sampling whereas the original DeepWalk paper used hierarchical sampling for the skip-gram model.

In [12]:
# p,q = 1 for DeepWalk as the random walks are completely unbiased. 
G = node2vec.Graph(user_product_graph, is_directed=False, p=1, q=1)

Compute the transition probabilities based on the edge weights. 

In [13]:
G.preprocess_transition_probs()

Compute the random walks. 10 walks for every node, each walk of length 80.

In [43]:
walks = G.simulate_walks(num_walks=10, walk_length=80)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [44]:
len(walks)

418450

Learn Embeddings via Gensim, which creates context/non-context pairs and then Skip-gram. Learn embeddings by optimizing the Skipgram objective using SGD. Uses Gensim Word2Vec.

In [46]:
def learn_embeddings(walks):
    
    walks = [list(map(str, walk)) for walk in walks]
    model = Word2Vec(walks, vector_size=50, window=10, min_count=0, sg=1, workers=8, epochs=1)
    return model.wv

In [47]:
node_embeddings = learn_embeddings(walks)

The output of gensim is a specific type of key-value pair with keys as the string-ed node ids and the values are numpy array of embeddings, each of shape (50,)

In [48]:
node_embeddings['0']

array([ 0.2357478 , -0.25867182, -0.2785079 ,  0.7711474 , -0.20884205,
        0.0565038 ,  0.17678171, -0.06870923,  0.55392593,  0.41653475,
       -0.13525803, -1.0533819 ,  0.09378741,  0.67649233, -0.42077023,
       -0.5922771 , -0.05365152, -0.3712859 ,  0.54637533, -1.3128067 ,
        0.7253521 ,  0.39309   ,  0.4593434 , -0.94637525,  0.39801636,
        0.08747377, -0.6898166 ,  0.27073142,  0.19203171,  0.2050887 ,
        0.24499173,  0.1625324 , -0.11759286, -0.01578157, -0.11571743,
       -0.41233498,  0.3490533 , -0.4721236 ,  0.2984353 , -0.21840766,
        0.08409699,  0.15944213,  0.3169213 ,  0.52808505,  0.33743402,
        0.45198315, -0.50526214,  0.25331646, -0.3361974 ,  0.51283467],
      dtype=float32)

In [49]:
node_embeddings['0'].shape

(50,)

Since we worked with integer ids for nodes, let's create reverse mapping dictionaries that map integer user/movie to their actual ids.

In [50]:
reverse_product2dict = {k:v for v,k in product2dict.items()}
reverse_user2dict = {k:v for v,k in user2dict.items()}

In [51]:
node_vecs = [node_embeddings[str(i)] for i in range(count)]
node_vecs = np.array(node_vecs)
node_vecs.shape

(41845, 50)

Top 5 products similar to a given movie as recommendations.

In [52]:
def get_similar_product_graph_embeddings(productid, product_embed, top_n=5):
    product_idx = product2dict[productid]
    query = product_embed[product_idx].reshape(1,-1)
    ranking = cosine_similarity(query, product_embed)
    top_ids = np.argsort(-ranking)[0]
    top_product_ids = [reverse_product2dict[j] for j in top_ids if j in reverse_product2dict][:top_n]
    sim_product = [id[0] for id in top_product_ids]
    return sim_product

In [87]:
productids = product2dict.keys()
dictionary = {}

for productid in productids:
    recommend = get_similar_product_graph_embeddings(productid, node_vecs)[:5]
    dictionary[productid[0]] = recommend

dictionary

{'0767905288': ['0767905288',
  '0767905296',
  '0553502689',
  '0553712438',
  '0375431268'],
 '1573229326': ['1573229326',
  '039914823X',
  '1573221937',
  '1573225517',
  '1573228214'],
 '006440823X': ['006440823X',
  '0807261521',
  '0064405176',
  '0786227737',
  '0590213113'],
 '0385504209': ['0385504209',
  '0739307312',
  '0375432302',
  '0739302043',
  '0671027352'],
 '0679444653': ['0679444653',
  '0060114185',
  '0060929790',
  '0072434236',
  '0375400699'],
 '0312966091': ['0312966091',
  '074351839X',
  '0684822652',
  '1578155444',
  '0312966970'],
 '0061007226': ['0061007226',
  '0451167538',
  '0451155750',
  '0451156609',
  '0425107469'],
 '0451163966': ['0451163966',
  '088103715X',
  '0453008151',
  '0140236015',
  '0670030589'],
 '0553801430': ['0553801430',
  '0553502719',
  '0553801376',
  '0553582755',
  '0739301748'],
 '0670030643': ['0670030643',
  '0786242930',
  '1565115465',
  '1565115457',
  '0142001805'],
 '1565113306': ['1565113306',
  '0393039765',
  '0

### Evaluation

In [97]:
data_small = copurchase_data.sample(n = 20)

In [98]:
len=0
i=0
lens_min=np.Inf
lens_max=0
for key, value in dictionary.items():
  for val in value:
    if nx.has_path(copurchaseGraph, source=key, target=val):
      lens=nx.shortest_path_length(copurchaseGraph, source=key, target=val)
      len+=lens
      i+=1
      if lens<lens_min:
        lens_min=lens
      if lens>lens_max:
        lens_max=lens

avg_len = len/i
print(avg_len)

2.338710151314823


The average length between actual purchase and our recommendation is 2.3

## Graph Embedding Using GCN

In [33]:
import networkx as nx
import scipy.sparse as sp
import torch

In [35]:
# Setup of Matrices

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float64)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def load_data():
    order = sorted(list(user_product_graph.nodes()))
    A = nx.to_scipy_sparse_matrix(user_product_graph, nodelist=order)
    I = sp.eye(A.shape[0])
    A_hat = A + I  # Adding Self-loops
    D = np.array(A_hat.sum(1))  # getting degree of all nodes
    D_inv = np.power(D, -0.5).flatten()
    D_inv[np.isinf(D_inv)] = 0
    D_inv = sp.diags(D_inv)
    A = sparse_mx_to_torch_sparse_tensor(A)
    I = sparse_mx_to_torch_sparse_tensor(I)
    A_hat = sparse_mx_to_torch_sparse_tensor(A_hat)
    D_inv = sparse_mx_to_torch_sparse_tensor(D_inv)
    A_old = torch.mul(A_hat, D_inv)
    A_new = torch.mul(D_inv, A_old)
    return I, G, A_new

In [36]:
# Create GCN Layers and GCN Model Step

import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.functional as F

class GraphConvolution(Module):

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.DoubleTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.DoubleTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        return output

    
class GCN(nn.Module):
    def __init__(self, in_features, out_features, hidden_layer_one_units, hidden_layer_two_units, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(in_features, hidden_layer_one_units)
        self.gc2 = GraphConvolution(hidden_layer_one_units, hidden_layer_two_units)
        self.gc3 = GraphConvolution(hidden_layer_two_units, out_features)
        self.dropout = dropout

    def forward(self, x, adj):    
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gc2(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc3(x, adj)
        return F.log_softmax(x, dim=1)

In [100]:
# Loss -> Optimizer -> Train

from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np
import networkx as nx
import scipy.sparse

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
def get_P_distribution(G):
    print("Getting P Distribution...")
    distances = []
    for i in sorted(list(G.nodes)):
        for j in sorted(list(G.nodes)):
            if i != j:

                temp_distance = nx.shortest_path_length(G, i, j)
                distances.append(temp_distance)
    
    mx = scipy.sparse.csr_matrix(distances)
    mx = sparse_mx_to_torch_sparse_tensor(mx)
    sum = torch.sparse.sum(mx)
    distances_norm = torch.div(mx,sum)
    distances_norm = Variable(distances_norm, requires_grad = True)
    return distances_norm

def get_Q_distribution(output):
    print("Getting Q Distribution...")
    distances = []
    for row_num1, node1 in enumerate(output):
        for row_num2, node2 in enumerate(output):
            if row_num1 != row_num2:
                temp_distance = torch.dist(node1, node2)
                distances.append(temp_distance.tolist())
    
    for i in range(0,len(distances)):
        if distances[i]==0:
            distances[i] = 0.000001
    mx = scipy.sparse.csr_matrix(distances)
    mx = sparse_mx_to_torch_sparse_tensor(mx)
    sum = torch.sparse.sum(mx)
    distances_norm = torch.div(mx,sum)
    return distances_norm

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float64)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


# Load data
I, G, A_new = load_data()
P = get_P_distribution(user_product_graph)

Getting P Distribution...


In [ ]:
# Model, Loss and optimizer
model = GCN(in_features = I.shape[1], out_features = 128, hidden_layer_one_units = 64, hidden_layer_two_units = 32, dropout = 0.5)
loss_fn = nn.KLDivLoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr = 0.01)


def get_old_Q_distribution(output):
    m = []
    for node1 in output:
        l = []
        for node2 in output:
            d = torch.dist(node1,node2)
            l.append(d.tolist())
        m.append(l)
    y = np.asarray(m)
    return(torch.from_numpy(np.matrix(y/y.sum(axis=1, keepdims = True))))
    
def train(epoch):
    t = time.time()
    # Forward Propagation
    model.train()
    optimizer.zero_grad()
    output = model.forward(I, A_new)
    Q = get_Q_distribution(output)
    loss = loss_fn(P.coalesce().values().log(),Q.coalesce().values())
    
    # Back Propagation
    loss = Variable(loss, requires_grad = True)
    print("loss=", loss)
    loss.backward()
    optimizer.step()
    return output

# Train model
t_total = time.time()
for epoch in range(2):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Train model
t_total = time.time()
I, G, A_new = load_data()
for epoch in range(2):
    output = train(epoch)
nodes = sorted(list(G.nodes))
output_embedding = output.tolist()
filename = "./GCN_Embeddings.txt"
result = list(zip(nodes,output_embedding))
with open(filename,'w') as f:
    for x in result:
        f.write(str(x[0]) + " ")
        for y in x[1]:
            f.write(str(y) + " ")
        f.write("\n")
f.close()
print("Optimization Finished successfully!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))